In [1]:
import numpy as np
import ipyparallel as ipp
%matplotlib inline

In [2]:
c = ipp.Client()
c.ids

[0, 1, 2, 3]

In [3]:
c[0:2] #no inlcuye el motor 2

<DirectView [0, 1]>

In [4]:
c[1:3].apply_sync(lambda: "Hello world")

C:\Users\AlbertoC\Anaconda2\lib\site-packages\dateutil\parser.py:605: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  elif res.tzname and res.tzname in time.tzname:


['Hello world', 'Hello world']

In [5]:
c[0].map_sync(lambda x,y,z: x+y+z, range(5), range(5), range(5))

[0, 3, 6, 9, 12]

### Función de apply_sync y map_sync
* apply_sync: Hace el mismo procedimiento en cada motor<br>
* map_sync:   Hace el procedimiento utilizando los motores en conjunto

In [6]:
x = 2**5
#range(int(x**0.5)+1)
#np.arange(int(x**0.5)+1) #calculos más rápidos con numpy
for i in np.arange(int(x**0.5)+1):
    print x % i

0
0
0
2
0
2


C:\Users\AlbertoC\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: RuntimeWarning: divide by zero encountered in long_scalars


In [7]:
%%timeit
x = 2**25
for i in np.arange(int(x**0.5)+1):
    x % i

C:\Users\AlbertoC\Anaconda2\lib\site-packages\ipykernel\__main__.py:259: RuntimeWarning: divide by zero encountered in long_scalars


1000 loops, best of 3: 1.34 ms per loop


### Instrucción %timeit
* %timeit: calcula el tiempo de la linea donde está la instrucción
* %%timeit: calcula el tiempo de la celda donde está la instrucción

all? 
Docstring:
all(iterable) -> bool

Return True if bool(x) is True for all values x in the iterable.
If the iterable is empty, return True.
Type:      builtin_function_or_method

In [8]:
PrimeQ = lambda x: all(x % i!=0 for i in np.arange(int (x**0.5)+1)[2:]) and x>1

In [9]:
def PrimeQ0(x):
    import numpy as np
    return all(x%i for i in np.arange(int(x**0.5)+1)[2:])

#### Nota
Preferiblemente llamar solo la función range al definir la función y no toda la libreria numpy
* Buscar cómo llamar solo la función

In [10]:
PrimeQ0(13)

True

In [11]:
PrimeQ0(2**7) #utiliza el kernel de python por defecto

False

In [12]:
c[0].apply_sync(PrimeQ0, 2**7) #utiliza una sola vista

False

In [13]:
dview = c[:]

In [14]:
2**np.arange(7)

array([ 1,  2,  4,  8, 16, 32, 64])

In [15]:
%timeit dview.map_sync(PrimeQ0, 2**np.arange(25))

10 loops, best of 3: 48.6 ms per loop


In [16]:
%%timeit 
for i in 2**np.arange(25):
    x % i

The slowest run took 8.89 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 12.2 µs per loop


In [17]:
np.array([x%i for i in np.arange(int(x**0.5)+1)[2:]])
#numpy es una representación eficiente de listas

array([0, 2, 0, 2])

In [18]:
np.all(np.array([x%i for i in np.arange(int(x**0.5)+1)[2:]]))
#numpy es una representación eficiente de listas

False

In [19]:
def PrimeQ1(x):
    import numpy as np
    return np.all(np.array([x%i for i in np.arange(int(x**0.5)+1)[2:]]))

In [36]:
%timeit dview.map_sync(PrimeQ1, 2**np.arange(25))

10 loops, best of 3: 47.1 ms per loop


Calculcar residuos creada con numpy utilizando todos los motores

In [21]:
np.mod?

### Aritmetica modular
Se hace sobre subconjuntos de números enteros<br>
El tiempo utiliza aritmética modular

In [22]:
np.mod(24,12)

0

In [23]:
np.arange(24)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

In [24]:
np.mod(np.arange(24),12)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,  0,  1,  2,  3,  4,
        5,  6,  7,  8,  9, 10, 11])

In [25]:
[hora % 12 for hora in np.arange(24)]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [26]:
np.array([hora % 12 for hora in np.arange(24)])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,  0,  1,  2,  3,  4,
        5,  6,  7,  8,  9, 10, 11])

In [27]:
#np.mod(np.arange(24),12)
#se sustituye el for, reduciendo el trabajo computacional
for hora in np.arange(24):
    print hora

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [28]:
for med in np.arange(12,20):
    print np.mod(np.arange(24), med)

[ 0  1  2  3  4  5  6  7  8  9 10 11  0  1  2  3  4  5  6  7  8  9 10 11]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12  0  1  2  3  4  5  6  7  8  9 10]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13  0  1  2  3  4  5  6  7  8  9]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14  0  1  2  3  4  5  6  7  8]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15  0  1  2  3  4  5  6  7]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16  0  1  2  3  4  5  6]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17  0  1  2  3  4  5]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18  0  1  2  3  4]


In [29]:
def PrimeQ2(x):
    import numpy as np
    lista = np.arange(np.int(x**0.5)+1)[2:]
    residuos = np.mod(x, lista)
    return np.all(residuos)

In [30]:
np.mod(12, np.arange(24)[2:])

array([ 0,  0,  0,  2,  0,  5,  4,  3,  2,  1,  0, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12])

implementando la misma operación con un for

In [31]:
[12 % i for i in np.arange(24)[2:]]

[0, 0, 0, 2, 0, 5, 4, 3, 2, 1, 0, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]

In [32]:
%timeit dview.map_sync(PrimeQ1, 2**np.arange(25))

10 loops, best of 3: 50.9 ms per loop


In [38]:
%timeit dview.map_sync(PrimeQ2, 2**np.arange(25))

10 loops, best of 3: 47.3 ms per loop


### Con bloqueo o sin bloqueo
* Bloqueo: espera hasta que todos los motores estén hechos ejecutando el comando<br>
      Dview.block = Verdadero
* No bloqueo: devuelve un AsyncResult de inmediato, obtiene datos más adelante con el método get, verifica el estado de check con listo<br>
      Dview.block = False
#### Con Apply
`apply_sync_`:(block=True)<br>
`apply_async_` :(block=False)